In [ ]:
# !pip install -q -U accelerate git+https://github.com/huggingface/peft.git
# !pip install -q datasets einops wandb
# !pip install -q bitsandbytes==0.43.2

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [5]:
import requests
import json

url = "https://90ca-140-203-228-110.ngrok-free.app/api/generate"

data = {
    'model': 'llama3.2-vision',
    'prompt':"hi"
}

response = requests.post(url,json = data)
response.text

'{"model":"llama3.2-vision","created_at":"2025-02-19T19:55:54.9592395Z","response":"Hello","done":false}\n{"model":"llama3.2-vision","created_at":"2025-02-19T19:55:55.2628976Z","response":"!","done":false}\n{"model":"llama3.2-vision","created_at":"2025-02-19T19:55:55.5477783Z","response":" Is","done":false}\n{"model":"llama3.2-vision","created_at":"2025-02-19T19:55:55.7775636Z","response":" there","done":false}\n{"model":"llama3.2-vision","created_at":"2025-02-19T19:55:55.9435867Z","response":" something","done":false}\n{"model":"llama3.2-vision","created_at":"2025-02-19T19:55:56.1088784Z","response":" I","done":false}\n{"model":"llama3.2-vision","created_at":"2025-02-19T19:55:56.2687449Z","response":" can","done":false}\n{"model":"llama3.2-vision","created_at":"2025-02-19T19:55:56.4245836Z","response":" help","done":false}\n{"model":"llama3.2-vision","created_at":"2025-02-19T19:55:56.7512666Z","response":" you","done":false}\n{"model":"llama3.2-vision","created_at":"2025-02-19T19:55:5

In [1]:
from datasets import load_dataset

dataset = load_dataset("Malikeh1375/medical-question-answering-datasets", "chatdoctor_healthcaremagic",split="train")
len(dataset)

112165

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
# model_name = "deepseek-r1:1.5b"
# model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
url = "https://90ca-140-203-2123-1230.ngrok-free.app/api/generate"
model_name= url + "deepseek-r1:1.5b"

# mod_nm = OllamaLLM(model=model_name)

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

url = "https://90ca-140-203-2123-1230.ngrok-free.app/api/generate"
model_name= url + "deepseek-r1:1.5b"
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             quantization_config = nf4_config,
                                             device_map = "auto",
                                             trust_remote_code = True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
from langchain_ollama import OllamaLLM

olama_model = OllamaLLM(model='deepseek-r1:1.5b')
olama_model

OllamaLLM(model='deepseek-r1:1.5b')

In [16]:
dataset.select(range(1000))

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 1000
})

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 16

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj"]
)

In [ ]:
# peft_config = LoraConfig(inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, peft_type=TaskType.CAUSAL_LM)
model = get_peft_model(model, peft_config)
model

In [ ]:
print(model.print_trainable_parameters())

In [ ]:
def format_dataset(data_point):
    prompt = f"""###SYSTEM: {data_point['instruction']}

    ###INPUT: {data_point['input']}

    ###Response: {data_point['output']}
    """
    tokens = tokenizer(prompt,
        truncation=True,
        max_length=1024,
        padding="max_length",)
    tokens["labels"] = tokens['input_ids'].copy()
    return tokens

In [ ]:
tokenized_dataset = dataset.map(format_dataset)

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(['input', "output","instruction"])
tokenized_dataset

In [ ]:
import torch
if torch.cuda.device_count() > 1: 
    model.is_parallelizable = True
    model.model_parallel = True

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    report_to="none",
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./lora_finetuned_model")

In [ ]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained("./lora_finetuned_model")

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, "./lora_finetuned_model")

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

In [ ]:
input_query = "I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.. Thank you doc!"
prompt = f"""
### 'instruction': "If you are a doctor, please answer the medical questions based on the patient's description."

### input : {input_query}

### output : 
"""

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model.generate(input_ids=input_ids, num_return_sequences=1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))